In [1]:
import pandas as pd
import re
import os
import email
import dateutil.parser as dateparser
import datetime

In [2]:
'''
(Note for AY)
Current issue with zipper-a: 
?
 -----Original Message-----
From: =09Mack, Iris =20
Sent:=09Wednesday, September 26, 2001 12:37 PM

shows -9114 days reply time. 
'''

'\n(Note for AY)\nCurrent issue with zipper-a: \n?\n -----Original Message-----\nFrom: =09Mack, Iris =20\nSent:=09Wednesday, September 26, 2001 12:37 PM\n\nshows -9114 days reply time. \n'

In [3]:
# Constants
# Tested and verified OK- small: lokey-t, south-s
user = "lokey-t" # NOTE: SOME PEOPLE DO NOT HAVE THE SAME REPLY FORMAT. 
send_folders = ["_sent_mail", "sent", "sent_items"] # do not edit this!

In [4]:
# Get the number of emails in user's folder. 
def get_num_sent_emails(user, foldername):
    n = (len(os.listdir("data/maildir/"+user+"/"+foldername)))
    return n

In [5]:
def get_user_email_id(user):
    send_folder = send_folders[0] # doesn't actually matter which folder we use.
    # get the "from" field of one of the sent emails
    sentfilename = os.listdir("data/maildir/" + user + "/" + send_folder)[0]
    fullfilename = "data/maildir/" + user + "/" + send_folder + "/" + sentfilename
    with open(fullfilename, "r") as f:
        for line in f:
            if line.startswith("From:"):
                return line.split(": ")[1].strip()
            

# print("Email ID for user {}: ".format(user), get_user_email_id(user))

In [6]:
## Make the dataframe, to be filled with data. 
df = pd.DataFrame(columns=["uid", "sender", "recipient", "sendtime", "replytime", \
                           "replyspeed", "messagecontent", "foldername"])
df.set_index("uid", inplace=True)
# Indices of the dataframe are messageIDs. (because properness.)

In [7]:
# Choose which sent folder to use.
sent_folder_name = ""
for folder in send_folders:
    if os.path.isdir("data/maildir/"+user+"/"+folder):
        if sent_folder_name == "":
            sent_folder_name = folder
        if get_num_sent_emails(user, folder) > get_num_sent_emails(user, sent_folder_name):
            sent_folder_name = folder


print("Using folder: ", sent_folder_name)


Using folder:  sent_items


In [8]:
def isReply(content):    
    # REGARDLESS of whether we have a fwd or a reply, we are looking for a block 
    # that looks like this:
    
    # SENDER on DATE 05/16/2001 05:30:21 PM
    # To: SOMEONE
    # cc: ??
    # Subject: xxx
    time_regex = "\d+:\d\d(|:\d\d)( |)(AM|PM)"
    date_slash_regex = "\d\d/\d\d/20\d\d " + time_regex
    whitespace_regex = r"[\s(=20)(=09)]+"
    a = re.search(date_slash_regex + whitespace_regex + r"To:", content)
    if a is not None:
        just_date = re.search(date_slash_regex, a.group()).group()
        return just_date
    
    # Format:
    # -----Original Message-----
    # From: whatever
    # Sent: Friday, October 05, 2001 3:07 PM
    day_regex = r"(\w+)day, \w+ \d\d, 20\d\d " + time_regex
    a = re.search(day_regex, content)
    if a is not None:
        return a.group()
        


    else:
        return -1



In [9]:
# Iterate through the sent emails and add to dataframe. 
filenames = os.listdir("data/maildir/"+user+"/"+sent_folder_name)
for file in filenames:
    full_filename = "data/maildir/"+user+"/"+sent_folder_name+"/"+file
    f = open(full_filename, "r")

    # Process the message
    message = email.message_from_file(f)
    f.close()

    # Get the list of recipients.
    # print(message.items())
    if message["To"]:
        recipient_list = message["To"].split(", ")
    if message["Cc"]:
        recipient_list += message["Cc"].split(", ")
    if message["Bcc"]:
        recipient_list += message["Bcc"].split(", ")
    # if message["X-To"]:
    #     recipient_list += message["X-To"].split(", ")
    # if message["X-cc"]:
    #     recipient_list += message["X-cc"].split(", ")
    # if message["X-bcc"]:
        recipient_list += message["X-bcc"].split(", ")
    recipient_list = list(set([r.strip() for r in recipient_list])) # remove duplicates
    if len(recipient_list) == 0:
        continue # Email not sent to anyone?

    # Get the time the email was sent.
    sendtime = dateparser.parse(message["Date"])
    # print("send time", sendtime.astimezone(datetime.timezone.utc))

    # Calculate the reply speed:
    prev_email_time = isReply(content=message.get_payload())
    if prev_email_time != -1:
        # print("reply time", prev_email_time)
        prev_email_time = dateparser.parse(prev_email_time + "+0000")
        # print("prev_email_time", prev_email_time.astimezone(datetime.timezone.utc))
        replyspeed = sendtime - prev_email_time
        # print("replyspeed", replyspeed)
    else:
        replyspeed = -1

    # print(message.get_payload()[:300])
    # print("---")

    # Make dataframe row(s):
    for recipient in recipient_list:
        row = {
            "uid": message["Message-ID"]+recipient,
            "sender": message["From"],
            "recipient": recipient,
            "sendtime": sendtime,
            "replytime": prev_email_time, 
            "replyspeed": replyspeed,
            "messagecontent": message.get_payload(),
            "foldername": ""
        }
        id = message["Message-ID"]+recipient,
        entry = pd.DataFrame([row])
        df = pd.concat([df, entry], ignore_index=True)

df

    


,sender,recipient,sendtime,replytime,replyspeed,messagecontent,foldername,uid
0,teb.lokey@enron.com,barbara.cain@enron.com,2001-09-26 13:58:24-07:00,2001-09-26 12:09:00+00:00,0 days 08:49:24,I have your shirt and will deliver it tomorrow...,,<3793068.1075858978961.JavaMail.evans@thyme>ba...
1,teb.lokey@enron.com,jeanette.doll@enron.com,2002-01-10 10:59:49-08:00,2002-01-10 12:36:00+00:00,0 days 06:23:49,Amen!\n\n -----Original Message-----\nFrom: \t...,,<31295237.1075861141181.JavaMail.evans@thyme>j...
2,teb.lokey@enron.com,kelly.allen@enron.com,2002-03-12 09:23:52-08:00,2002-03-12 10:48:00+00:00,0 days 06:35:52,Please plan to attend also. It involves trans...,,<30360337.1075861142494.JavaMail.evans@thyme>k...
3,teb.lokey@enron.com,rachel.cady@enron.com,2002-01-11 11:46:49-08:00,2002-01-11 13:46:00+00:00,0 days 06:00:49,Looks great to me.\n\n -----Original Message--...,,<2590085.1075861141289.JavaMail.evans@thyme>ra...
4,teb.lokey@enron.com,sheila.nacey@enron.com,2002-01-11 11:46:49-08:00,2002-01-11 13:46:00+00:00,0 days 06:00:49,Looks great to me.\n\n -----Original Message--...,,<2590085.1075861141289.JavaMail.evans@thyme>sh...
...,...,...,...,...,...,...,...,...
362,teb.lokey@enron.com,doug.cebryk@enron.com,2001-10-22 07:25:14-07:00,2001-10-22 09:15:00+00:00,0 days 05:10:14,Please add Blair Lichtenwalter to all Sun Devi...,,<32860459.1075858978310.JavaMail.evans@thyme>d...
363,teb.lokey@enron.com,johnny.mcgee@enron.com,2001-10-22 07:25:14-07:00,2001-10-22 09:15:00+00:00,0 days 05:10:14,Please add Blair Lichtenwalter to all Sun Devi...,,<32860459.1075858978310.JavaMail.evans@thyme>j...
364,teb.lokey@enron.com,eric.gadd@enron.com,2001-10-22 07:25:14-07:00,2001-10-22 09:15:00+00:00,0 days 05:10:14,Please add Blair Lichtenwalter to all Sun Devi...,,<32860459.1075858978310.JavaMail.evans@thyme>e...
365,teb.lokey@enron.com,david.sztroin@enron.com,2002-03-06 13:33:36-08:00,2002-03-06 15:33:00+00:00,0 days 06:00:36,Thanks!\n\n -----Original Message-----\nFrom: ...,,<18093427.1075861142228.JavaMail.evans@thyme>d...


In [10]:
# SAVE AS CSV
!mkdir -p processed_data
df.to_csv("processed_data/"+user+"_sent.csv")